In [1]:
import pdfplumber
import pandas as pd
import numpy as np
import os
import textEx
from datetime import datetime
import re

In [2]:
def readAllXingYingPDF(dir:str) -> pd.DataFrame:
    all_pages_df = pd.DataFrame()

    for filename in os.listdir(dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(dir, filename)

            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    
                    table = page.extract_table()
                    if table:
                        df = pd.DataFrame(table[1:])
                        df=df.dropna()
                        df.insert(1, 'date', filename)
                        
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
    all_pages_df=all_pages_df.drop(columns=[1])
    all_pages_df = all_pages_df[all_pages_df[0]!='']
    all_pages_df['仓位'] = all_pages_df[0].astype(str).str.split().str[-1]
    all_pages_df['到取价'] = all_pages_df[0].astype(str).str.split().str[-2]
    all_pages_df['规格'] = all_pages_df[0].astype(str).str.split().str[-3]
    all_pages_df['国家'] = all_pages_df[0].astype(str).str.split().str[1]
    all_pages_df['貨名'] = [' '.join(sublist) for sublist in all_pages_df[0].astype(str).str.split().str[2:-3]]
    all_pages_df=all_pages_df.iloc[1:,:]

    return all_pages_df

In [3]:
all_pages_df=readAllXingYingPDF('quotation\盈星')

In [4]:
all_pages_df = all_pages_df[all_pages_df['国家'].notna()]
all_pages_df = all_pages_df[~all_pages_df[0].str.contains('月')]
all_pages_df = all_pages_df[~all_pages_df[0].str.contains('訂價')]
all_pages_df = all_pages_df[all_pages_df['规格']!='頁，共']
all_pages_df = all_pages_df.reset_index(drop=True)
for i in range(1,len(all_pages_df)):
    if re.match(r'\d+', all_pages_df['国家'][i]):
        print(i)
        try:
            all_pages_df['国家'][i] = all_pages_df['貨名'][i].split().str[0]
        except:
            print(all_pages_df['貨名'])
all_pages_df = all_pages_df.reset_index(drop=True)

In [8]:
def _specToList(spec :list) -> list:

    # Sort 'spec' in alphabetical order
    spec.sort()

    # Initialize variables with empty strings
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""

    # Assign values from 'spec' to 'spec1' to 'spec6' based on the length of 'spec'
    if len(spec) > 0:
        spec1 = spec[0]
    if len(spec) > 1:
        spec2 = spec[1]
    if len(spec) > 2:
        spec3 = spec[2]
    if len(spec) > 3:
        spec4 = spec[3]
    if len(spec) > 4:
        spec5 = spec[4]
    if len(spec) > 5:
        spec6 = spec[5]

    return [spec1,spec2,spec3,spec4,spec5,spec6]
        

def getYingXingQuote(df: pd.DataFrame) -> pd.DataFrame:
    
    df['specs'] = df['貨名'].apply(textEx.getSpec)
    df['brand'] = df['貨名'].apply(textEx.getBrand)
    for index, row in all_pages_df.iterrows():
        if row['brand'] is not None:
            all_pages_df.at[index, 'productName'] = row['貨名'].replace(row['brand'], '')
    df['productTag'] = df['貨名'].apply(textEx.getProduct)
    df['supplier'] = '盈星'
    df['category'] = df['貨名'].apply(textEx.getCategory)
    df['packing']=df['貨名'].apply(textEx.getPacking)
    df['origin'] = df['国家'].apply(textEx.getCountry)
    df['effectiveDate'] = df['date'].str.replace(' 現貨報價單.pdf', '')
    df[['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6']] = df['specs'].apply(lambda x: pd.Series(_specToList(x)))
    df['price'] = df['到取价'].map(lambda x: textEx.getPrice(x))
    df['weightUnit'] = df['规格'].astype(str).apply(textEx.getWeightUnit)
    df['warehouse']=df['仓位'].apply(textEx.getWarehoue)
    df['notes']= None

    df = df.explode('warehouse')
    df['warehouse'] = df['warehouse'].astype(str)

    required_columns = [
    'productName', 'productTag', 'supplier', 'category', 'packing',
    'origin', 'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3',
    'spec4', 'spec5', 'spec6', 'price', 'weightUnit', 'warehouse', 'notes'
    ]
    df=df[required_columns]
    return df

In [9]:
df=getYingXingQuote(all_pages_df)

c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not

...

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\tradeasy\textEx.py:63: UserWarning: DataFrame columns are not

No valid price found in: '/'
no _match is found
no _match is found
no _match is found


In [10]:
df.to_csv("yingxing.csv")